# Constructing TensorFlow Recommenders Model

In [1]:
import tensorflow_recommenders as tfrs

2023-06-14 17:04:13.310779: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import pandas as pd
import numpy as np

In [3]:
import string

In [4]:
fiverr_df = pd.read_csv('/Users/rsoedarnadi/Documents/GitHub/machine-learning/fiverr_data.csv', delimiter=',')

In [5]:
fiverr_df.head(6)

,illustrator_name,gig_rating,gig_price_from,gig_img_url,gig_url,gig_design_type,gig_design_style,client_origins,client_names,illustrator_origins
0,nadia_younus,5.0,60,https://fiverr-res.cloudinary.com/t_gig_cards_...,https://www.fiverr.com/nadia_younus/do-minimal...,logos,minimalist,Canada,stoxventures,Pakistan
1,mariya_logo99,4.9,15,https://fiverr-res.cloudinary.com/t_gig_cards_...,https://www.fiverr.com/mariya_logo99/do-creati...,logos,minimalist,Italy,lifehossain,Bangladesh
2,mountdesign,4.9,10,https://fiverr-res.cloudinary.com/t_gig_cards_...,https://www.fiverr.com/mountdesign/design-3-pr...,logos,minimalist,United States,stormproofrooof,India
3,ahmed_gfx786,4.9,5,https://fiverr-res.cloudinary.com/t_gig_cards_...,https://www.fiverr.com/ahmed_gfx786/design-a-u...,logos,minimalist,France,remi_bgd,Pakistan
4,websutra,4.9,400,https://fiverr-res.cloudinary.com/t_gig_cards_...,https://www.fiverr.com/websutra/draw-an-unique...,logos,minimalist,United States,tribolumen,Sri Lanka
5,kiara43,4.9,5,https://fiverr-res.cloudinary.com/t_gig_cards_...,https://www.fiverr.com/kiara43/create-modern-m...,logos,minimalist,Germany,sustainifytoday,Sri Lanka


In [6]:
fiverr_df = fiverr_df.dropna().drop_duplicates().reset_index(drop=True)

In [9]:
from typing import Dict, Text

import tensorflow as tf
import tensorflow_recommenders as tfrs

In [10]:
ratings = tf.data.Dataset.from_tensor_slices(dict(fiverr_df[['gig_img_url','client_names', 'gig_rating', 'gig_design_type', 'gig_design_style']]))
imgs = tf.data.Dataset.from_tensor_slices(dict(fiverr_df[['gig_img_url']]))

In [11]:
ratings = ratings.map(lambda x: {
    "gig_img_url": x["gig_img_url"],
    "client_names": x["client_names"],
    "gig_rating": float(x["gig_rating"]),
    "gig_design_type": x["gig_design_type"],
    "gig_design_style": x["gig_design_style"]
})

imgs = imgs.map(lambda x: x["gig_img_url"])

In [12]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(35_406, seed=42, reshuffle_each_iteration=False)
train = ratings.take(30_000)
test = ratings.skip(30_000).take(5_406)

In [13]:
img_urls = imgs.batch(1_000)
clients_names = ratings.batch(1_000_000).map(lambda x: x['client_names'])

unique_img_urls = np.unique(np.concatenate(list(img_urls)))
unique_clients_names = np.unique(np.concatenate(list(clients_names)))

unique_img_urls[:10]

2023-06-14 17:06:34.570718: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1686]
	 [[{{node Placeholder/_0}}]]
2023-06-14 17:06:34.619700: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [1686]
	 [[{{node Placeholder/_2}}]]


array([b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/100175685/original/0531df1a9873ba7d4bfa28831916295c8996b355.png',
       b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/100267701/original/dc68a5ca86285ba97ed2fdbac7eb8118fb79b129.jpg',
       b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/100633290/original/d2e5623413c4650fd54972f6accc5ed33c5d8144.jpg',
       b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/100752175/original/1e8500b760079bbeb8d57e8d09c979fb6cb6a478.jpg',
       b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/100849695/original/861139cd50b2e53d18e301ae12619c2d397ee171.jpg',
       b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/100955264/original/0500c21e89e1349e2cea19cdcb7f64c9dede9ba3.jpg',
       b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/101019442/original/e532854981e67059236c848215363fc0d5d175f

In [14]:
embedding_dimension = 32

client_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary=unique_clients_names, mask_token=None),
        tf.keras.layers.Embedding(len(unique_clients_names) + 1,   
                                  embedding_dimension)
])

item_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary=unique_img_urls, mask_token=None),
        tf.keras.layers.Embedding(len(unique_img_urls) + 1, 
                                  embedding_dimension)
])

In [15]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=imgs.batch(128).map(item_model)
)

task = tfrs.tasks.Retrieval(
  metrics=metrics
)


In [16]:
class Model(tfrs.Model):
  def __init__(self, client_model, item_model):
    super().__init__()
    self.item_model: tf.keras.Model = item_model
    self.client_model: tf.keras.Model = client_model
    self.retrieval_task: tf.keras.layers.Layer = task
    
  
  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    client_embeddings = self.client_model(features['client_names'])
    positive_item_embeddings = self.item_model(features['gig_img_url'])

    # The task computes the loss and the metrics.

    return self.retrieval_task(client_embeddings, positive_item_embeddings)

In [17]:
tf.keras.backend.clear_session()

In [18]:
model = Model(client_model, item_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [19]:
cached_train = train.shuffle(1_686).batch(300).cache()
cached_test = test.batch(300).cache()

In [20]:
model.fit(cached_train, epochs=10)

Epoch 1/10


2023-06-14 17:06:59.731730: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype double and shape [1686]
	 [[{{node Placeholder/_4}}]]


2/6 [=========>....................] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0050 - factorized_top_k/top_10_categorical_accuracy: 0.0050 - factorized_top_k/top_50_categorical_accuracy: 0.0300 - factorized_top_k/top_100_categorical_accuracy: 0.0683 - loss: 1711.0682 - regularization_loss: 0.0000e+00 - total_loss: 1711.0682 

2023-06-14 17:07:01.734425: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1686]
	 [[{{node Placeholder/_0}}]]
2023-06-14 17:07:01.815339: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype resource
	 [[{{node Placeholder/_6}}]]
2023-06-14 17:07:01.899077: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype resource
	 [[{{node Placeholder/_8}}]

5/6 [========================>.....] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0027 - factorized_top_k/top_10_categorical_accuracy: 0.0040 - factorized_top_k/top_50_categorical_accuracy: 0.0267 - factorized_top_k/top_100_categorical_accuracy: 0.0607 - loss: 1711.1087 - regularization_loss: 0.0000e+00 - total_loss: 1711.1087

...

6/6 [==============================] - 1s 90ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.9994 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 1278.4628 - regularization_loss: 0.0000e+00 - total_loss: 1278.4628
Epoch 10/10
2/6 [=========>....................] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 1353.9510 - regularization_loss: 0.0000e+00 - total_loss: 1353.9510

2023-06-14 17:07:07.626431: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1686]
	 [[{{node Placeholder/_0}}]]
2023-06-14 17:07:07.734938: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype float and shape [300,32]
	 [[{{node Placeholder/_12}}]]
2023-06-14 17:07:07.823862: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype resource
	 [[{{node

5/6 [========================>.....] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.9993 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 1353.6410 - regularization_loss: 0.0000e+00 - total_loss: 1353.6410

2023-06-14 17:07:07.920833: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype float and shape [300,32]
	 [[{{node Placeholder/_12}}]]
2023-06-14 17:07:08.003836: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype resource
	 [[{{node Placeholder/_10}}]]
2023-06-14 17:07:08.090610: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype resource
	 [[{{node Placeholder/

6/6 [==============================] - 1s 92ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.9994 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 1178.5671 - regularization_loss: 0.0000e+00 - total_loss: 1178.5671


In [28]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.client_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((imgs.batch(100), imgs.batch(100).map(model.item_model)))
)

# Get recommendations.
_, gigs = index(tf.constant(["ismetrix"]))
print(f"Recommendations for user: {gigs[0, :10]}")

Recommendations for user: [b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/106187992/original/12afe768e2ba9af5f211d0358dff19f6e419d38d.jpg'
 b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/157708259/original/6d07c8ffc8711b9a13ae4b2a54066a64b9cb1e99.jpg'
 b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/284476072/original/6dc3faf88a9da51f3e4972df0cf8501f06fb6589.jpg'
 b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/288420404/original/d1c877d2eae76ef2401e21e0a2de7a4b18c262c2.png'
 b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/288420404/original/d1c877d2eae76ef2401e21e0a2de7a4b18c262c2.png'
 b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/154184609/original/51d50b5af09a79fbc309048d4506ea6f701a30d5.jpg'
 b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/240161600/original/3420d134e48a95c626815837a051de5e42c258fe.jpg'
 b'https://five

2023-06-14 17:44:31.592328: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1686]
	 [[{{node Placeholder/_0}}]]
2023-06-14 17:44:31.593141: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_14' with dtype resource
	 [[{{node Placeholder/_14}}]]


In [29]:
import tempfile
import os

In [33]:
# Export the query model.
with tempfile.TemporaryDirectory() as tmp:
  path = os.path.join(tmp, "model")

  # Save the index.
  tf.saved_model.save(index, path)

  # Load it back; can also be done in TensorFlow Serving.
  loaded = tf.saved_model.load(path)

  # Pass a user id in, get top predicted movie titles back.
  scores, gigs = loaded(["ismetrix"])

  print(f"Recommendations: {gigs[0][:3]}")

INFO:tensorflow:Assets written to: /var/folders/ht/1vg09v7x16x7sbqnmzfvc4600000gn/T/tmphjao8i48/model/assets


INFO:tensorflow:Assets written to: /var/folders/ht/1vg09v7x16x7sbqnmzfvc4600000gn/T/tmphjao8i48/model/assets


Recommendations: [b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/106187992/original/12afe768e2ba9af5f211d0358dff19f6e419d38d.jpg'
 b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/157708259/original/6d07c8ffc8711b9a13ae4b2a54066a64b9cb1e99.jpg'
 b'https://fiverr-res.cloudinary.com/t_gig_cards_web,q_auto,f_auto/gigs/284476072/original/6dc3faf88a9da51f3e4972df0cf8501f06fb6589.jpg']


In [ ]:
tf.saved_model.save(index, "tfrs-model")